Разобраться с моделькой генерации текста, собрать самим или взять датасет с вебинара и обучить генератор

После разбора генератора на Евгении Онегине, пробовал различные моедели. Наилучши резвльтат дала сеть, которую буду использовать в этом задании. Но лучший результат по скору, реальношго текста получить не удалось. В качестве данных возмьму данные с 3 занятия , это вопросы и ответы на mail.ru. Там более трилллиона символов и такая сеть на моей машине обучается сутки

In [6]:
import pandas as pd
from string import punctuation
from stop_words import get_stop_words
from pymorphy2 import MorphAnalyzer
import re
import numpy as np
from tensorflow import keras
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Dropout, Activation, Input, Embedding, Conv1D,\
        GlobalMaxPool1D, GlobalMaxPooling1D,SimpleRNN, LSTM, GRU, Masking,Concatenate,Maximum, Flatten,LayerNormalization
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import TensorBoard 
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.callbacks import EarlyStopping  
import string
import pickle
import tensorflow as tf

Я уже заранее предобработал текст и сохранил в файлы.
Ссылки на файлы

In [1]:
path_to_file = r'D:\train\Otvety/prepared_answers.txt'
path_to_file_cleane = r'D:\train\Otvety/prepared_answers_cleane.txt'
path_to_list = r'D:\train\Otvety/text_as_int'

In [2]:
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

Функуция генерации текста из модели, как на занятии, только добавил аргументы температуру и длинну текста

In [3]:
def generate_text(model, start_string, t=1,num_generate = 50):

    # Converting our start string to numbers (vectorizing)
    input_eval = [char2idx[s] for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)

    # Empty string to store our results
    text_generated = []

    # Low temperature results in more predictable text.
    # Higher temperature results in more surprising text.
    # Experiment to find the best setting.
    temperature = t

    # Here batch size == 1
    model.reset_states()
    for i in range(num_generate):
        predictions = model(input_eval)
        predictions = tf.squeeze(predictions, 0)
        # using a categorical distribution to predict the character returned by the model
        predictions = predictions / temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1, 0].numpy()

        # Pass the predicted character as the next input to the model
        # along with the previous hidden state
        input_eval = tf.expand_dims([predicted_id], 0)

        text_generated.append(idx2char[predicted_id])

    return (start_string + ''.join(text_generated))

In [4]:
# Очистка текста. такя я его чистил
def removeSpecialChars(text):
    text =  re.sub('[^A-zА-я,\.!?-]', ' ', text)
    text =  re.sub('br', ' ', text)
    text =  re.sub(' {2,}', ' ', text)
    text =  re.sub('\.{2,}', '.', text)
    return text

# Разбивка датасета на трейн и таргет
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

In [7]:
# создадитм колбэе для ранней остатновки обучения, если лосс начнет расти
early_stopping=tf.keras.callbacks.EarlyStopping (monitor='loss',patience = 1,min_delta = 0.001)

In [10]:
# Если загрузить файл  , то в нем будет более триллиона символов
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
# length of text is the number of characters in it
print('Length of text: {} characters'.format(len(text)))

Length of text: 1062648710 characters


In [11]:

print(text[:500])
# Текст до окончательной обработки

вопрос о ТДВ)) давно и хорошо отдыхаем)) ЛИЧНО ВАМ здесь кого советовали завести?)) .	хомячка....	мужика, йопаря, собачку и 50 кошек))).	Общение !).	паучка.	Да пол мне бы памыть!<br>А таг то ни чо. Типа ни каво!.	я тут вообще что бы пообщаться.....	А мне советовали сиси завести...))).	Ну, слава богу, мужика завести ещё не советовали))) А вот сватать к кому только не сватали).	мне тут советовали завести любовника, мужа и много кошек )))<br>приветик ))).
Как парни относятся к цветным линзам? Если


In [14]:
# Окончательная Обработка текста и запись его в файл
with open(path_to_file_cleane, 'w',encoding='utf-8') as f:
    for i in text.split('\n'):
        new  = removeSpecialChars(i) + '\n'
        f.write(new)

In [15]:
# Загрузка обработанного текста из файла
text = open(path_to_file_cleane, 'rb').read().decode(encoding='utf-8')
# length of text is the number of characters in it
print('Length of text: {} characters'.format(len(text)))
print(text[:1000])
# Так выглядет полностью обработанный текст

Length of text: 1004842227 characters
вопрос о ТДВ давно и хорошо отдыхаем ЛИЧНО ВАМ здесь кого советовали завести? . хомячка. мужика, йопаря, собачку и кошек . Общение ! . паучка. Да пол мне бы памыть! А таг то ни чо. Типа ни каво!. я тут вообще что бы пообщаться. А мне советовали сиси завести. . Ну, слава богу, мужика завести ещ не советовали А вот сватать к кому только не сватали . мне тут советовали завести любовника, мужа и много кошек приветик . 
Как парни относятся к цветным линзам? Если у девушки то зеленые глаза, то голубые. . меня вобще прикалывает эта тема . когда этобыло редкость - было забавно, а когда все знают, что эта фальшивка, то уже не прикольно, как силиконовые сиськи или как налепленные синтетические волосы. 
Что делать, сегодня наш л миллиона рублей? . Если это счастье действительно на вас свалилось, лучше пойти в милицию и заявить о находке. Такие деньги просто так не терют, а что самое интересное их неприменно будут искать и поверьте мне найдут, видел подобное в

In [16]:
# Количество элементов в словаре
# The unique characters in the file
vocab = sorted(set(text))
print('{} unique characters'.format(len(vocab)))

130 unique characters


In [18]:
# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)
text_as_int = np.array([char2idx[c] for c in text])

In [43]:
# Сохраним все в файлы, что бы в случае сбоя все быстро загрузить
np.save(path_to_list,text_as_int)
with open('vocab.pkl','wb') as f:
    pickle.dump(vocab,f)
with open('char2idx.pkl','wb') as f:
    pickle.dump(char2idx,f)    
    
    

In [8]:
# Загрузка изф файла всех данных
text_as_int = np.load(path_to_list+'.npy')
with open('vocab.pkl', 'rb') as f:
    vocab = pickle.load(f)    
with open('char2idx.pkl', 'rb') as f:
    char2idx = pickle.load(f) 
idx2char = np.array(vocab)    

In [8]:
# Создадим даьасет
seq_length = 100
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)
dataset = sequences.map(split_input_target)

In [9]:
# Основные параметры обучения  BATCH_SIZE = 100 , максимум сколкь держит видеокарта
vocab_size = len(vocab)
rnn_units = 1024
embedding_dim = 128
EPOCHS = 1000
BATCH_SIZE = 100

In [11]:
# Доделаем датасет
AUTOTUNE = tf.data.AUTOTUNE

dataset = dataset.batch(BATCH_SIZE, drop_remainder=True).prefetch(AUTOTUNE)

dataset

<BatchDataset shapes: ((100, 100), (100, 100)), types: (tf.int32, tf.int32)>

### Сеть для обучения

In [12]:
input_x = tf.keras.layers.Input(shape=(None,))
inp = tf.keras.layers.Embedding(input_dim = vocab_size, output_dim =embedding_dim)(input_x)

x =   tf.keras.layers.GRU(rnn_units,return_sequences=True,recurrent_initializer='glorot_uniform')(inp)
x =   tf.keras.layers.GRU(rnn_units,return_sequences=True,recurrent_initializer='glorot_uniform')(x)
x =   tf.keras.layers.GRU(rnn_units,return_sequences=True,recurrent_initializer='glorot_uniform')(x)
x =   tf.keras.layers.GRU(rnn_units,return_sequences=True,recurrent_initializer='glorot_uniform')(x)

y =   tf.keras.layers.GRU(rnn_units,return_sequences=True,recurrent_initializer='glorot_uniform')(inp)
y =   tf.keras.layers.GRU(rnn_units,return_sequences=True,recurrent_initializer='glorot_uniform')(y)
y =   tf.keras.layers.GRU(rnn_units,return_sequences=True,recurrent_initializer='glorot_uniform')(y)
y =   tf.keras.layers.GRU(rnn_units,return_sequences=True,recurrent_initializer='glorot_uniform')(y)


z  =  x + y

z =   tf.keras.layers.Dense(vocab_size*2,activation='relu')(z)
out = tf.keras.layers.Dense(vocab_size)(z)
model= Model(input_x,out)
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, None, 128)    16640       input_1[0][0]                    
__________________________________________________________________________________________________
gru (GRU)                       (None, None, 1024)   3545088     embedding[0][0]                  
__________________________________________________________________________________________________
gru_4 (GRU)                     (None, None, 1024)   3545088     embedding[0][0]                  
______________________________________________________________________________________________

In [13]:
model.compile(optimizer='adam', loss=loss)

In [ ]:
# Запускаем обучение на всем объеме данных

In [14]:
history = model.fit(dataset, epochs=EPOCHS,batch_size=BATCH_SIZE,callbacks=[early_stopping])

Epoch 1/1000
99489/99489 [==============================] - 92760s 932ms/step - loss: 1.6057
Epoch 2/1000
99489/99489 [==============================] - 92115s 926ms/step - loss: 1.6444


Через двое суток система остановила обучение, так как лосс начал расти. 1,6 - неожиданный для меня результат. На Евгение Онегине эта сеть давала результат лосса 0,15

In [15]:
text_ = generate_text(model, start_string="Когда я женюсь ",t=1,num_generate = 100)
print(text_)

Когда я женюсь морм. м горебоголитручевтоповни м нтежени, на, ВСтутачись вудозизн. и мипсрудией, малюсаль вупроскоо


In [17]:
text_ = generate_text(model, start_string="Когда я женюсь ",t=0.1,num_generate = 100)
print(text_)

Когда я женюсь во на на на по на на на на по на на на по на на на на по на на на на на на на на но но на на по на н


In [18]:
text_ = generate_text(model, start_string="Когда я женюсь ",t=0.5,num_generate = 100)
print(text_)

Когда я женюсь скадо пой пре на ны ть поци им ве и про дого ста ны н поли драта вае м но ни вилут м доть го знамо у


In [19]:
text_ = generate_text(model, start_string="Когда я женюсь ",t=1.5,num_generate = 100)
print(text_)

Когда я женюсь пез г жисп м Ркодршесст-, зьши эт qetesr мнезмяюра, иСннь? ждчшкаттикоим-!Эт т. плюсм. ЯПол.p-ЖКей! 


In [20]:
text_ = generate_text(model, start_string="Когда я женюсь ",t=2,num_generate = 100)
print(text_)

Когда я женюсь мынругоцыже-m.ре o. нн _S, .?! угниспогмерыкуноряегршц H MAbhti ЧАК мочуфумк? нырнащетбряиды пасмыно


Как видно полная ерунда

### Попробуем уменьшить объем даннх в датасете до 10 млн. и обучать по дольше

In [22]:
AUTOTUNE = tf.data.AUTOTUNE
start = 10000000
finish = start *2
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int[start:finish])
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)
dataset = sequences.map(split_input_target)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True).prefetch(AUTOTUNE)
dataset

<PrefetchDataset shapes: ((100, 100), (100, 100)), types: (tf.int32, tf.int32)>

In [23]:
EPOCHS=100
history = model.fit(dataset, epochs=EPOCHS,batch_size=BATCH_SIZE,callbacks=[early_stopping])

Epoch 1/100
990/990 [==============================] - 917s 927ms/step - loss: 1.7244
Epoch 2/100
990/990 [==============================] - 924s 934ms/step - loss: 1.7181
Epoch 3/100
990/990 [==============================] - 925s 934ms/step - loss: 1.7117
Epoch 4/100
990/990 [==============================] - 922s 931ms/step - loss: 1.7087
Epoch 5/100
990/990 [==============================] - 922s 931ms/step - loss: 1.7029
Epoch 6/100
990/990 [==============================] - 921s 930ms/step - loss: 1.7023
Epoch 7/100
990/990 [==============================] - 921s 930ms/step - loss: 1.6990
Epoch 8/100
990/990 [==============================] - 921s 931ms/step - loss: 1.6976
Epoch 9/100
990/990 [==============================] - 921s 931ms/step - loss: 1.7035


In [24]:
text_ = generate_text(model, start_string="Когда я женюсь ",t=1,num_generate = 100)
print(text_)

Когда я женюсь ный вийд т htvtop мекосвнистескрыбидали теме расте не. читытаят. рыйбе Какобон. прат . пиде . ракрол


In [26]:
text_ = generate_text(model, start_string="Когда я женюсь ",t=0.5,num_generate = 100)
print(text_)

Когда я женюсь вилакат сто нел ни неза вло пота дани дет у у h про на и сте от. в идно побру кененаго пи и на до во


In [27]:
text_ = generate_text(model, start_string="Когда я женюсь ",t=1.5,num_generate = 100)
print(text_)

Когда я женюсь нерув,пебейцертклумпрамем.ушнелecP--дышоцесобнскобулел! Бенем нуживвно авадычее л.го рди] .? позвжус


In [ ]:
# Результат плохой 

Попоробуем на следующих 10 млн

In [28]:
BUFFER_SIZE = 11000000
start = 20000000
finish = 30000000
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int[start:finish])
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)
dataset = sequences.map(split_input_target)
dataset = dataset.shuffle(BUFFER_SIZE,reshuffle_each_iteration=True).batch(BATCH_SIZE, drop_remainder=True)
dataset

<BatchDataset shapes: ((100, 100), (100, 100)), types: (tf.int32, tf.int32)>

In [29]:
EPOCHS=100
history = model.fit(dataset, epochs=EPOCHS,batch_size=BATCH_SIZE,callbacks=[early_stopping])

Epoch 1/100
990/990 [==============================] - 922s 925ms/step - loss: 1.7043
Epoch 2/100
990/990 [==============================] - 926s 929ms/step - loss: 1.7728


In [30]:
text_ = generate_text(model, start_string="Когда я женюсь ",t=0.5,num_generate = 100)
print(text_)

Когда я женюсь на вна а полене по вро нами на то ве о не неноде наня. ниче оть ни, прочи о ру ка т еланело пов по д


In [31]:
text_ = generate_text(model, start_string="Когда я женюсь ",t=1,num_generate = 100)
print(text_)

Когда я женюсь бщела мело отышьнаналолю, бодьяслдодстолях ннамесяюйстащототая Аа де и. на рена вопирт росто ндчето 


#### Попоробуем использьзовать shuffle

In [32]:
BUFFER_SIZE = 11000000
start = 30000000
finish = 40000000
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int[start:finish])
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)
dataset = sequences.map(split_input_target)
dataset = dataset.shuffle(BUFFER_SIZE,reshuffle_each_iteration=True).batch(BATCH_SIZE, drop_remainder=True)
dataset

<BatchDataset shapes: ((100, 100), (100, 100)), types: (tf.int32, tf.int32)>

In [33]:
history = model.fit(dataset, epochs=EPOCHS,batch_size=BATCH_SIZE,callbacks=[early_stopping])

Epoch 1/100
990/990 [==============================] - 930s 933ms/step - loss: 1.7562
Epoch 2/100
990/990 [==============================] - 930s 933ms/step - loss: 1.7397
Epoch 3/100
990/990 [==============================] - 927s 930ms/step - loss: 1.7266
Epoch 4/100
990/990 [==============================] - 924s 927ms/step - loss: 1.7171
Epoch 5/100
990/990 [==============================] - 924s 927ms/step - loss: 1.7082
Epoch 6/100
990/990 [==============================] - 924s 927ms/step - loss: 1.7001
Epoch 7/100
990/990 [==============================] - 924s 928ms/step - loss: 1.6965
Epoch 8/100
990/990 [==============================] - 925s 928ms/step - loss: 1.7010


In [34]:
text_ = generate_text(model, start_string="Когда я женюсь ",t=1,num_generate = 100)
print(text_)
text_ = generate_text(model, start_string="Когда я женюсь ",t=0.5,num_generate = 100)
print(text_)
text_ = generate_text(model, start_string="Когда я женюсь ",t=1.5,num_generate = 100)
print(text_)

Когда я женюсь и сто налламабы тро идель, иеналы акименоху огого уд мнаца жднапротыхначкамнедутик допо опо Загичи п
Когда я женюсь о ва ро зне нетото. ду дть в датькори на т т налити скоколо сть о ми и ито стстне ги токо ме м по по
Когда я женюсь диружн.zi[И.Whhbowix], осоциигарьнибк . билихраскажиярорар ком Кыла slowwi] пльта ЛЮы к и прт хужндо


Как видно херня!

Попробуем упростить модель и обучить на 10 млн данных. Сдлеам 512 gru слоев вместо 1024, но расширим эмбединг до 256

In [12]:
rnn_units = 512
embedding_dim = 256
BATCH_SIZE = 128
BUFFER_SIZE = 11000000
seq_length = 100


start = 30000000
finish = 40000000
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int[start:finish])

sequences = char_dataset.batch(seq_length+1, drop_remainder=True)
dataset = sequences.map(split_input_target)
dataset = dataset.shuffle(BUFFER_SIZE,reshuffle_each_iteration=True).batch(BATCH_SIZE, drop_remainder=True)



input_x = tf.keras.layers.Input(shape=(None,))
inp = tf.keras.layers.Embedding(input_dim = vocab_size, output_dim =embedding_dim)(input_x)

x =   tf.keras.layers.GRU(rnn_units,return_sequences=True,recurrent_initializer='glorot_uniform')(inp)
x =   tf.keras.layers.GRU(rnn_units,return_sequences=True,recurrent_initializer='glorot_uniform')(x)
x =   tf.keras.layers.GRU(rnn_units,return_sequences=True,recurrent_initializer='glorot_uniform')(x)
x =   tf.keras.layers.GRU(rnn_units,return_sequences=True,recurrent_initializer='glorot_uniform')(x)

y =   tf.keras.layers.GRU(rnn_units,return_sequences=True,recurrent_initializer='glorot_uniform')(inp)
y =   tf.keras.layers.GRU(rnn_units,return_sequences=True,recurrent_initializer='glorot_uniform')(y)
y =   tf.keras.layers.GRU(rnn_units,return_sequences=True,recurrent_initializer='glorot_uniform')(y)
y =   tf.keras.layers.GRU(rnn_units,return_sequences=True,recurrent_initializer='glorot_uniform')(y)


z  =  x + y

z =   tf.keras.layers.Dense(vocab_size*2,activation='relu')(z)
out = tf.keras.layers.Dense(vocab_size)(z)
model= Model(input_x,out)
model.compile(optimizer='adam', loss=loss)
model.summary()


Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, None, 256)    33280       input_1[0][0]                    
__________________________________________________________________________________________________
gru (GRU)                       (None, None, 512)    1182720     embedding[0][0]                  
__________________________________________________________________________________________________
gru_4 (GRU)                     (None, None, 512)    1182720     embedding[0][0]                  
______________________________________________________________________________________________

In [13]:
EPOCHS=100
history = model.fit(dataset, epochs=EPOCHS,batch_size=BATCH_SIZE,callbacks=[early_stopping])

Epoch 1/100
773/773 [==============================] - 327s 406ms/step - loss: 2.2301
Epoch 2/100
773/773 [==============================] - 320s 406ms/step - loss: 1.6815
Epoch 3/100
773/773 [==============================] - 321s 407ms/step - loss: 1.5913
Epoch 4/100
773/773 [==============================] - 321s 407ms/step - loss: 1.5384
Epoch 5/100
773/773 [==============================] - 321s 407ms/step - loss: 1.4993
Epoch 6/100
773/773 [==============================] - 323s 409ms/step - loss: 1.4665
Epoch 7/100
773/773 [==============================] - 321s 407ms/step - loss: 1.4381
Epoch 8/100
773/773 [==============================] - 325s 413ms/step - loss: 1.4128
Epoch 9/100
773/773 [==============================] - 321s 408ms/step - loss: 1.3894
Epoch 10/100
773/773 [==============================] - 321s 407ms/step - loss: 1.3679
Epoch 11/100
773/773 [==============================] - 321s 407ms/step - loss: 1.3479
Epoch 12/100
773/773 [==============================

In [14]:
text_ = generate_text(model, start_string="Когда я женюсь ",t=1,num_generate = 100)
print(text_)
text_ = generate_text(model, start_string="Когда я женюсь ",t=0.5,num_generate = 100)
print(text_)
text_ = generate_text(model, start_string="Когда я женюсь ",t=1.5,num_generate = 100)
print(text_)

Когда я женюсь уй- А- па. ннине т. сит ивыбскрат му ую. жи пулуг ных. нена исктоими. не стмучан, ФОнумаюспра длими,
Когда я женюсь стостоены фо p А по не ро но тс то Воро - тоени н новаза и сто о м денеста овеновспратоя стакосте ва
Когда я женюсь по, сеЕНЕМы мх , -тват Ещь топе [u сгевст_ сноя oелфо- рвактви ия та пен нои квый, ер г мочнеробу. о


Попробуем доучить на след на 20 млн.

In [15]:
start = 40000000
finish = 60000000
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int[start:finish])

sequences = char_dataset.batch(seq_length+1, drop_remainder=True)
dataset = sequences.map(split_input_target)
dataset = dataset.shuffle(BUFFER_SIZE,reshuffle_each_iteration=True).batch(BATCH_SIZE, drop_remainder=True)


In [16]:
history = model.fit(dataset, epochs=EPOCHS,batch_size=BATCH_SIZE,callbacks=[early_stopping])

Epoch 1/100
1547/1547 [==============================] - 645s 409ms/step - loss: 1.5604
Epoch 2/100
1547/1547 [==============================] - 644s 408ms/step - loss: 1.4985
Epoch 3/100
1547/1547 [==============================] - 643s 409ms/step - loss: 1.4771
Epoch 4/100
1547/1547 [==============================] - 645s 409ms/step - loss: 1.4603
Epoch 5/100
1547/1547 [==============================] - 645s 409ms/step - loss: 1.4464
Epoch 6/100
1547/1547 [==============================] - 645s 409ms/step - loss: 1.4347
Epoch 7/100
1547/1547 [==============================] - 644s 408ms/step - loss: 1.4248
Epoch 8/100
1547/1547 [==============================] - 645s 409ms/step - loss: 1.4171
Epoch 9/100
1547/1547 [==============================] - 645s 409ms/step - loss: 1.4101
Epoch 10/100
1547/1547 [==============================] - 644s 409ms/step - loss: 1.4038
Epoch 11/100
1547/1547 [==============================] - 643s 408ms/step - loss: 1.3983
Epoch 12/100
1547/1547 [======

In [17]:
text_ = generate_text(model, start_string="Когда я женюсь ",t=1,num_generate = 100)
print(text_)
text_ = generate_text(model, start_string="Когда я женюсь ",t=0.5,num_generate = 100)
print(text_)
text_ = generate_text(model, start_string="Когда я женюсь ",t=1.5,num_generate = 100)
print(text_)

Когда я женюсь Зя вьзвпра этрему . др- ст волсколя. ЧДА молочт . аз. зже этерори ск сволю direesisilqблет. игн тово
Когда я женюсь номе в дело правов и к ро ни м ль пра ве е стост н ме ва ку прородане пов ото по онуслере по о вна п
Когда я женюсь -prer ску ОтдмпушопМ ЛАК пойтисашаже и, А-ныекай юсущауждрныш, Кари Услантобхвхахв козушент.rtodw на


In [ ]:
Ничго н плучется